<a href="https://colab.research.google.com/github/gorillaion/Research/blob/master/M_Series_response.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from scipy import interpolate
from scipy.integrate import odeint

In [2]:
class MseriesGeneratorClass:
    def __init__(self, k, Tc, binary=[0,1]):
        self._generate_vec = np.vectorize(self._generate)
        
        comb = np.array([
            [2, 1], [3, 1], [4, 1], [5, 2], [6, 2], [7, 1],
            [7, 3], [10,3], [17,3], [18,7], [21,2], [22,1]])
        
        (self._p, self._q) = comb[k]
        self._Tc = Tc
        self._T = 2**self._p - 1
        self._binary = binary
        self._mseries = [1] + [0 for i in range(self._p-1)]
        for i in range(self._p, self._T):
            self._mseries.append(self._mseries[i-self._p] ^ self._mseries[i-self._q])
        self._mseries = np.array(self._mseries)
        print(f"p = {self._p}, q = {self._q}, T = {self._T}")
    
    def get_period(self):
        return self._T
    
    def get_mseries(self):
        return self._mseries
    
    def _generate(self, t):
        k = int(t/self._Tc)
        if k > self._T-1:
            k = k % self._T
        else:
            pass #nothing
        u = self._mseries[k]
        if u > 0:
            u = self._binary[1]
        else:
            u = self._binary[0]
        return u
    
    def output(self, t):
        if type(t) == float:
            u = self._generate(t)
        else:
            u = self._generate_vec(t)
        return u


class PlantClass:
    def __init__(self):
        # ++ Parameter ++
        I1 = 9.38062e-4 # [kgm^2] ....... 慣性モーメント（振子部）
        I2 = 6.56e-4    # [kgm^2] ....... 慣性モーメント（ロータ部）
        m1 = 0.669      # [kg] .......... 振子部の質量（モータを含む）
        m2 = 0.519      # [kg] .......... ロータ部の質量
        lg = 0.059758   # [m] ........... 主軸 - 重心間の距離
        l  = 0.157	# [m] ........... 軸間距離（主軸 - ロータ軸）
        c1 = 0.02       # [Nm.sec/rad] .. 粘性摩擦トルク係数（第1軸）
        c2 = 0.009      # [Nm.sec/rad] .. 粘性摩擦トルク係数（第2軸）
        Kt = 2.07e-3    # [Nm/A] ........ トルク定数
        Ka = 1          # [--] .......... DCアンプゲイン
        Rm = 2.8	# [Ω] ........... 抵抗
        gr = 1/51       # [--] .......... 減速比
        g  = 9.81       # [m/s^2] ....... 重力加速度

        I = I1 + m1 * lg**2 + m2 * l**2
        b = (Kt * Ka) / (gr * Rm)
        f = (m1 * lg + m2 * l) * g / I
        b1 = b/I
        b2 = b/I + b/I2

        # ++ State Space Matrix ++
        self._A = np.array([
            [0, 0, 1, 0],
            [0, 0, 0, 1],
            [0, 0,-c1/I, c2/I],
            [0, 0, c1/I, -(c2/I + c2/I2)]
        ])
        self._B = np.array([0, 0, -b1, b2])
        self._F = np.array([0, 0, -f, f])

        #self._Al = np.array([        # Linearization of A
        #    [   0, 0, 1, 0],
        #    [   0, 0, 0, 1],
        #    [-f/I, 0,-c1/I, c2/I],
        #    [ f/I, 0, c1/I, -(c2/I + c2/I2)]])
        
    def act(self, x, u):
        return self._A.dot(x) + self._B*u + self._F*np.sin(x[0])


In [3]:
work_path = "/content/drive/My Drive/Colab Notebooks"


# 立ち上がり時間の測定

In [ ]:
step_x = pd.read_csv(f"{work_path}/data/step_x.csv")
step_x2 = step_x[["x2[k]"]]
step_x4 = step_x[["x4[k]"]]

rsp100 = step_x4[19000:].mean()[0]
rsp90 = rsp100 * 0.9
rsp10 = rsp100 * 0.1

print(f"max:{rsp100: 8.4f}")
print(f"90%:{rsp90: 8.4f}")
print(f"10%:{rsp10: 8.4f}") # x4[7 < k < 8]


In [14]:
step_x4_spline_t = interpolate.interp1d(step_x4.values[:,0], step_x["Time[sec]"].values, kind="quadratic")

t_rsp10 = step_x4_spline_t(rsp10)
t_rsp90 = step_x4_spline_t(rsp90)

In [ ]:
y = step_x4.values[:,0]
t = step_x["Time[sec]"].values

fig, ax = plt.subplots(1,1)
ax.plot(t, y)
ax.hlines([rsp100, rsp90, rsp10], -0.05, 0.2, color=["r","g","b"], linestyle=":")
ax.vlines([t_rsp90, t_rsp10], 0, 50, color=["g","b"])
ax.set_xlim(-0.01,0.2);


In [ ]:
tau = t_rsp90 - t_rsp10
Tc_lower = tau/10

print(f"立ち上がり時間: {tau:.4f} (Tc > {Tc_lower:.4f})")

# M系列信号の入力に対する時刻歴応答の取得

In [ ]:
# define parameters
Tc  = 0.0165

# create object
plant = PlantClass()
gen = MseriesGeneratorClass(7, Tc, binary=[-12.0, 12.0])

print(f"The time of one period: {Tc*gen.get_period():.3f}[sec]")

# define control-system
def system(x, t):
    u = gen.output(t)
    return plant.act(x, u)

In [24]:
Ts  = 0.001
sim_time = 20.0+Ts
t_span = np.arange(0, sim_time, Ts)

In [27]:
# run the simulation
resp_x = odeint(system, [0.,0.,0.,0.], t_span)
ref_u = gen.output(t_span)

dxdt = []
for i in range(ref_u.size):
    dxdt.append(plant.act(resp_x[i],ref_u[i]))
dxdt = np.array(dxdt)

data = np.hstack((resp_x, dxdt[:,2:], ref_u[:,np.newaxis]))

In [ ]:
# check the results
plt.subplot(211)
plt.plot(t_span, resp_x[:,0])

plt.subplot(212)
plt.plot(t_span, ref_u)

In [ ]:
df_respx = pd.DataFrame({
    "Time[sec]"  : np.round(t_span[:-1],3),
    "x1[k]"      : data[:-1, 0],
    "x2[k]"      : data[:-1, 1],
    "x3[k]"      : data[:-1, 2],
    "x4[k]"      : data[:-1, 3],
    "d/dt x3[k]" : data[:-1, 4],
    "d/dt x4[k]" : data[:-1, 5],
    "voltage"    : data[:-1, 6]
})

df_respy = pd.DataFrame({
    "Time[sec]" : np.round(t_span[1:],3),
    "x1[k+1]"   : data[1:, 0],
    "x2[k+1]"   : data[1:, 1],
    "x3[k+1]"   : data[1:, 2],
    "x4[k+1]"   : data[1:, 3]
})

In [33]:
data_label = "ms3a"

df_respx.to_csv(f"{work_path}/data/{data_label}_x.csv", index=False)
df_respy.to_csv(f"{work_path}/data/{data_label}_y.csv", index=False)